In [1]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import re

In [2]:
import requests
import urllib.request
import pandas as pd
import csv
from bs4 import BeautifulSoup

# Web Scraping Data From Wikipedia
Most cities in the data set have very informative Wikipedia Sites. This data is publicaly avialable to anyone with a internet connection. As we're are particular interested in the comparing the CO2 emision with climate/geological data we're partilcuar interested in the section on Wikipedia page which describe a city's cliamte.  Below can a screenshot be seen of the Climate Data table for Berlin. 
![title](Berlin_climate.png)
Similar tables exisist for almost all the cities in the dataset. This data can be gatehered using the `BeautifulSoup` library. 

In [3]:
df = pd.read_csv(os.path.abspath('../data/Processed/Cities_imputed.csv'), index_col=0)

In [4]:
climate_df = df[['City', 'Country', 'Continent']]

In [5]:
climate_df['wiki_link'] = [re.sub(r'\([^)]*\)', '', city).replace(" ", "_") for city in climate_df.City]

In [6]:
climate_df.loc[climate_df['wiki_link'] == 'Birmingham']

,City,Country,Continent,wiki_link
319,Birmingham(AL),United States,North America,Birmingham
261,Birmingham,United Kingdom,Europe,Birmingham


## Correcting for the name of the city does land on the right Wikipage

In [7]:
def update_wiki_link(df, city, city_update):
    if city in df.wiki_link.values:
        df.loc[df['wiki_link'] == city, 'wiki_link'] = city_update
    else:
        print(city)

In [8]:
cities_updates = [
    ('Santa_Cruz', 'Santa_Cruz_(Bolivia)'),
    ('Cordoba' , 'Córdoba,_Argentina'),
    ('San_Francisco_Bay_Area', 'San_Fransisco'),
    ('Tampa-St._Petersburg', 'Saint_Petersburg_(Florida)'),
    ('Memphis', 'Memphis,_Tennessee'),
    ('Denver-Aurora', 'Aurora,_Colorado'),
    ('Rotterdam-Hague', 'Rotterdam'),
    ('Nashville-Davidson', 'Nashville,_Tennessee'),
    ('Cologne-Bonn', 'Cologne'),
    ('Richmond', 'Richmond,_Virginia'),
    ('Rochester', 'Rochester,_New_York'),
    ('Phoenix-Mesa', 'Phoenix_(Arizona)'),
    ('Leon', 'León,_Guanajuato'),
    ('Buffalo', 'Buffalo,_New_York'),
    ('Allentown-Bethlehem', 'Allentown,_Pennsylvania'),
    ('San_Juan', 'San_Juan,_Puerto_Rico'),
    ('Puebla', 'Puebla_(city)'),
    ('San_Jose', 'San_José,_Costa_Rica'),
    ('Dallas-Fort_Worth', 'Fort_Worth,_Texas'),
    ('Seoul-Incheon', 'Seoul'),
    ('Lome', 'Lomé'),
    ('Chihuahua', 'Chihuahua_City'),
    ('Fukuoka-Kitakyushu', 'Fukuoka'),
    ('Davao', 'Davao_City'),
    ('New_York', 'New_York_City'),
    ('Osaka-Kobe-Kyoto', 'Osaka'),
    ('Kansas_City', 'Kansas_City,_Missouri'),
    ('Minneapolis-St._Paul', 'Minneapolis'),
    ('Portland', 'Portland,_Oregon'),
    ('Phoenix_(Arizona)', 'Phoenix,_Arizona'),
    ('Washington', 'Washington,_D.C.'),
    ('Columbus', 'Columbus,_Ohio'),
    ('Salvador', 'Salvador,_Bahia'),
    ('Xi‚Äôan', 'Xi%27an'),
    ('Concepcion', 'Concepción,_Chile'),
    ('Providence', 'Providence,_Rhode_Island')
]

for city, city_update in cities_updates:
    update_wiki_link(
        df = climate_df,
        city = city,
        city_update = city_update
    )

In [9]:
climate_df.loc[(climate_df['wiki_link'] == 'Birmingham') & (climate_df['Country'] == 'United States')] = 'Birmingham,_Alabama'
climate_df.loc[(climate_df['wiki_link'] == 'Valencia') & (climate_df['Country'] == 'United Venezuela')] = 'Valencia,_Carabobo'

In [10]:
def extract_climate_table(soup):
    # exstract the table with the cliamte data
    tables = soup.findAll('table',{"class":"collapsible"})

    select_table = None
    for table in tables:
        header = table.findAll(['th'])[0].getText()[0:12].lower()
        
        if 'climate' in header:
            select_table = table
            break
    
    return select_table

In [12]:
climate_data_dict = dict()

for wiki_link in climate_df.wiki_link:
    url = 'https://en.wikipedia.org/wiki/' + wiki_link

    response = requests.get(url)
    status_code = response.status_code
    if status_code != 200:
        print('Could not find city: ', wiki_link, 'Wikipedia')
        continue

    soup = BeautifulSoup(response.text,"html.parser")

    # exstract the table with the cliamte data
    table = extract_climate_table(soup)

    # In case the city does not have a climate table
    if not table:
        climate_data_dict[wiki_link] = None
        print('No climate data available for city: ', wiki_link)
        continue

    trs = table.findAll(['tr'])[1:]

    for i, cell in enumerate(trs):
        th = cell.find_all('th')
        th_data = [col.text.strip('\n') for col in th]  
        td = cell.find_all('td')
        row = [i.text.replace('\n','') for i in td]
        
        
        if i == 0:
            columns = th_data
            city_climate_df_tmp = pd.DataFrame(columns=columns)
        else:
            
            if len(th_data) == 0:
                continue
                
            tmp = pd.DataFrame({key: [value] for key, value in zip(columns, th_data+row)})
            city_climate_df_tmp = city_climate_df_tmp.append(tmp)

    climate_data_dict[wiki_link] = city_climate_df_tmp
    features = climate_data_dict[wiki_link]['Month'].values
    #print(len(features), '  ', features)


No climate data available for city:  Cebu
No climate data available for city:  Caracas
No climate data available for city:  Santo_Domingo
No climate data available for city:  Monterrey


## Processing the climate data

In [18]:
climate_data_dict['Copenhagen']

,Month,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
0,Record high °C (°F),11.8(53.2),15.8(60.4),20.8(69.4),26.2(79.2),28.5(83.3),32.7(90.9),33.0(91.4),33.8(92.8),29.8(85.6),23.2(73.8),16.6(61.9),12.8(55.0),33.8(92.8)
0,Average high °C (°F),3.4(38.1),3.6(38.5),6.5(43.7),11.8(53.2),16.7(62.1),19.6(67.3),22.2(72.0),21.8(71.2),17.5(63.5),12.6(54.7),7.6(45.7),4.4(39.9),12.3(54.1)
0,Daily mean °C (°F),1.4(34.5),1.4(34.5),3.5(38.3),7.7(45.9),12.5(54.5),15.6(60.1),18.1(64.6),17.7(63.9),13.9(57.0),9.8(49.6),5.5(41.9),2.5(36.5),9.1(48.4)
0,Average low °C (°F),−0.7(30.7),−0.8(30.6),0.7(33.3),4.2(39.6),8.6(47.5),11.9(53.4),14.3(57.7),14.1(57.4),10.8(51.4),7.1(44.8),3.3(37.9),0.5(32.9),6.2(43.2)
0,Record low °C (°F),−26.3(−15.3),−20.0(−4.0),−18.5(−1.3),−8.8(16.2),−3.4(25.9),1.0(33.8),0.7(33.3),0.6(33.1),−3.2(26.2),−7.0(19.4),−15.2(4.6),−16.0(3.2),−26.3(−15.3)
0,Average precipitation mm (inches),53.0(2.09),36.9(1.45),42.3(1.67),35.8(1.41),47.2(1.86),63.9(2.52),60.9(2.40),67.5(2.66),61.0(2.40),63.3(2.49),56.4(2.22),57.4(2.26),645.7(25.42)
0,Average precipitation days (≥ 0.1 mm),14.9,11.4,13.5,11.5,10.8,12.0,12.4,12.0,13.6,14.5,15.4,15.4,157.4
0,Average snowy days,5.9,4.4,4.1,1.3,0.0,0.0,0.0,0.0,0.0,0.2,1.7,3.9,21.4
0,Average relative humidity (%),86,84,82,76,72,72,73,75,78,83,84,85,79
0,Mean monthly sunshine hours,51.5,68.1,119.7,180.9,230.2,213.3,228.1,198.9,141.9,100.9,55.3,40.6,"1,629.7"


In [38]:
city_climate_list = [list(climate_df.Month.values) for city, climate_df in climate_data_dict.items() if climate_df is not None]
set_of_climate_variable = set([city for sublist in city_climate_list for city in sublist])

In [39]:
flat_list

{'Average afternoon relative humidity (%)',
 'Average dew point °C (°F)',
 'Average dew point °F (°C)',
 'Average high °C (°F)',
 'Average high °F (°C)',
 'Average low °C (°F)',
 'Average low °F (°C)',
 'Average precipitation days',
 'Average precipitation days (≥ 0.005 inch)',
 'Average precipitation days (≥ 0.01 in)',
 'Average precipitation days (≥ 0.01 in.)',
 'Average precipitation days (≥ 0.01 inch)',
 'Average precipitation days (≥ 0.01\xa0mm)',
 'Average precipitation days (≥ 0.1 mm)',
 'Average precipitation days (≥ 0.1\xa0mm)',
 'Average precipitation days (≥ 0.2 mm)',
 'Average precipitation days (≥ 0.2\xa0mm)',
 'Average precipitation days (≥ 0.5 mm)',
 'Average precipitation days (≥ 1.0 mm)',
 'Average precipitation days (≥ 1\xa0mm)',
 'Average precipitation inches (mm)',
 'Average precipitation mm (inches)',
 'Average rainfall inches (mm)',
 'Average rainfall mm (inches)',
 'Average rainy days',
 'Average rainy days (≥ 0.01 in)',
 'Average rainy days (≥ 0.01 inch)',
 'Ave

In [ ]:
rename = {
    
} 

In [12]:
def seperate_metrix_from_imperial(row, th_data):
    rainfall = ['Average precipitation mm (inches)', 'Average rainfall inches (mm)', 'Average rainy days', 'Average precipitation inches (mm)', 'Average rainfall mm (inches)']

    matric = []
    for imperial_metric in row:
        try:
            val1, val2 = imperial_metric.replace(")", "").split('(')
            
            # Remove symbol which look like a minus sign which is not not a minus sign
            val1, val2 = val1.replace('−', '-'), val2.replace('−', '-')
            # Remove commas which seperates tree zeros and convert to float 
            val1, val2 = float(val1.replace(',', '')), float(val2.replace(',', ''))

            # if rainfall when select minimeters
            if th_data in rainfall:
                matric.append(max(val1, val2))
            # else it is a temperature select celcius
            else:
                matric.append(min(val1, val2))
        except ValueError:
            try:
                matric.append(float(imperial_metric))
            except ValueError:
                matric.append(None)
    return matric

In [13]:
focus_climate_data = [
    'Record high °C (°F)', 'Record high °F (°C)',
    'Average high °C (°F)', 'Average high °F (°C)', 
    'Daily mean °C (°F)', 'Daily mean °F (°C)', 'Daily mean °C (°F)',
    'Average low °C (°F)', 'Average low °F (°C)',
    'Record low °C (°F)', 'Record low °F (°C)',
    'Average precipitation mm (inches)', 'Average rainfall inches (mm)', 'Average rainy days', 'Average precipitation inches (mm)', 'Average rainfall mm (inches)',
    'Average precipitation days (≥ 0.1 mm)', 'Average rainy days (≥ 0.01 in)', 'Average precipitation days (≥ 1.0 mm)', 'Average precipitation days (≥ 0.5 mm)', 'Average precipitation days (≥ 0.01 in)', 'Average rainy days (≥ 0.1 mm)'
]

climate_data_names = [
    'Record high °C (°F)', 
    'Average high °C (°F)', 
    'Daily mean °C (°F)',
    'Average low °C (°F)', 
    'Record low °C (°F)', 
    'Average precipitation mm (inches)', 
    'Average precipitation days (≥ 0.1 mm)',
]